In [1]:
import sys

In [2]:
sys.version

'3.14.0rc1 free-threading build (v3.14.0rc1:48f88310044, Jul 22 2025, 13:19:27) [Clang 16.0.0 (clang-1600.0.26.6)]'

In [3]:
sys._is_gil_enabled()

False

In [6]:
import concurrent.futures
from functools import partial

import numpy as np

In [7]:
def path_distance(ij, adj_mat: np.ndarray, max_dist=5) -> int:
    """Find the path distance between nodes `i` and `j` on a graph described by an adjacency matrix, `adj_mat`.
    Give up after `max_dist` edge traversals.

    Returns
    -------
    int
        Number of steps (less than or equal to `max_dist`) between nodes `i` and `j`.
        Returns -1 if no path found within the allowed number of steps.
    """
    i, j = ij
    if i == j:
        return 0
    step = 1
    connections = adj_mat[i, :]
    # import pdb; pdb.set_trace()
    while step < max_dist:
        if connections[j] > 0:
            return step
        step += 1
        connections = connections @ adj_mat
    return -1

In [8]:
M = int(5e4)
# M = 10
N = M * M
np.random.seed(0)
a1 = (np.random.rand(M, M) < 0.0005).astype(np.uint8)

In [9]:
# Adjacency matrix takes up 2.5 GB or memory
a1.nbytes / 1e9

2.5

In [10]:
%%time
path_distance((5, 10), a1)

CPU times: user 15.7 s, sys: 42.4 ms, total: 15.7 s
Wall time: 15.7 s


4

In [11]:
%%time
path_distance((4, 15), a1)

CPU times: user 16 s, sys: 44.7 ms, total: 16.1 s
Wall time: 16.1 s


4

In [13]:
%%time
f = partial(path_distance, adj_mat=a1)
with concurrent.futures.ThreadPoolExecutor() as p:
    distances_par_tasks = [p.submit(f, (i, j)) for i in range(4) for j in range(4)]
    distances_par_results = [t.result() for t in distances_par_tasks]

CPU times: user 9min 46s, sys: 2.04 s, total: 9min 48s
Wall time: 51.6 s


In [15]:
dict(sorted(zip(*np.unique(distances_par_results, return_counts=True)), key=lambda tup: tup[1], reverse=True))

{np.int64(4): np.int64(10), np.int64(0): np.int64(4), np.int64(3): np.int64(2)}